In [1]:
!ngrok authtoken 33fuuG4Ngki9PvDNZtuJR8WOMEO_7SKmxmP41c8AaHrKKrGd7

Authtoken saved to configuration file: C:\Users\elici\AppData\Local/ngrok/ngrok.yml


In [12]:
%%writefile mylibrary.py

import streamlit as st

def render_cart(cart):
    subtotal = 0
    for item in cart.values():
        st.image(item["image"], width=100)
        st.write(f"{item['name']}: {item['qty']} × ${item['price']:.2f}")
        subtotal += item["qty"] * item["price"]
    return subtotal

def apply_discount(code, subtotal, valid_codes):
    value = valid_codes[code] * subtotal
    return -value, f"Discount applied: -${value:.2f}"

def check_free_shipping(subtotal_after_discount, threshold=100):
    if subtotal_after_discount >= threshold:
        return 0, "🎉 You’ve unlocked free shipping!", 1.0
    else:
        remaining = threshold - subtotal_after_discount
        progress = max(0, min(subtotal_after_discount / threshold, 1.0))
        return 5, f"Spend ${remaining:.2f} more to unlock free shipping", progress

def calculate_gst(amount, rate=0.08):
    return amount * rate

def calculate_total(subtotal, discount, gst, shipping):
    return subtotal + discount + gst + shipping

def load_saved_card(filepath="card_details.txt"):
    card = {}
    try:
        with open(filepath, "r") as f:
            for line in f:
                if "|" in line:
                    key, value = line.strip().split("|", 1)
                    card[key] = value
    except FileNotFoundError:
        pass
    return card

def save_card_details(card, filepath="card_details.txt"):
    with open(filepath, "w") as f:
        for key, value in card.items():
            f.write(f"{key}|{value}\n")


def render_card_payment(saved_card):
    use_saved = bool(saved_card) and st.checkbox("Use saved card details")

    if use_saved:
        st.markdown("#### 💳 Using saved card:")
        masked = "**** **** **** " + saved_card.get("Card Number", "")[-4:]
        st.write(f"**Card Number:** {masked}")

    card_fields = [
        ("Card Number", "card_number"),
        ("Cardholder Name", "card_name"),
        ("Expiry Date (MM/YY)", "card_expiry"),
        ("CVV", "card_cvv", "password")
    ]

    card = {}
    for field in card_fields:
        label, key = field[0], field[1]
        field_type = field[2] if len(field) > 2 else "default"

        default = saved_card.get(label, "") if use_saved else ""
        card[key] = st.text_input(label, value=default, key=key,
                                  type="password" if field_type == "password" else "default")

    if not use_saved and st.checkbox("Save this card for future use"):
        save_card_details({f[0]: card[f[1]] for f in card_fields})

    return card



Overwriting mylibrary.py


In [1]:
%%writefile payment.py

import streamlit as st
import mylibrary

# Set page config
st.set_page_config(page_title="Payment")

# Simulate a pre-filled cart
# To be replaced with the cart code
if 'cart' not in st.session_state:
    st.session_state.cart = {
        "capybara": {
            "name": "Capybara Plush",
            "price": 25.00,
            "qty": 3,
            "image": "images/capybara.jpg"
        }, 
        "snoopy": {
            "name": "Snoopy Keychain",
            "price": 12.00,
            "qty": 3,
            "image": "images/snoopy.jpg"
        }
    }

# Layout: Left = Contact, delivery & payment options, Right = Order Summary
col1, col2 = st.columns([2, 1])

# Right Col: Order Summary
with col2:
    st.subheader("🛒 Order Summary")
    # To be replaced with cart code
    subtotal = mylibrary.render_cart(st.session_state.cart)

    # Discount state tracking
    st.session_state.setdefault("applied_discount_code", "")
    st.session_state.setdefault("discount_value", 0)
    
    # Valid discount codes
    valid_codes = {"FRIEND10": 0.10, "FAMILY20": 0.20}
    
    # Discount code input
    code = st.text_input("Discount code or gift card")
    applied_code = st.session_state.applied_discount_code
    previously_applied = applied_code in valid_codes
    
    # Always define discount before using it
    discount = st.session_state.discount_value
    
    # Discount logic
    if code in valid_codes:
        if previously_applied:
            st.warning("Only one discount code can be applied. Your discount has been updated to the current code.")
        discount, msg = mylibrary.apply_discount(code, subtotal, valid_codes)
        st.session_state.discount_value = discount
        st.session_state.applied_discount_code = code
        st.success(msg)
    elif code:
        st.error("Invalid discount code")

    # Free shipping logic
    shipping_cost, shipping_msg, progress = mylibrary.check_free_shipping(subtotal + discount)
    st.write(shipping_msg)
    if shipping_cost == 0:
        st.success("🎉 You’ve unlocked free shipping!")
    else:
        st.progress(progress)

    # GST and total
    gst = mylibrary.calculate_gst(subtotal + discount)
    total = mylibrary.calculate_total(subtotal, discount, gst, shipping_cost)

    # Final summary
    st.write(f"Subtotal: ${subtotal:.2f}")
    st.write(f"Discount: ${discount:.2f}")
    st.write(f"GST (8%): ${gst:.2f}")
    st.write(f"Shipping: ${shipping_cost:.2f}")
    st.subheader(f"Total: ${total:.2f}")

# Left Col: Contact & Payment
with col1:
#  Contact Information
    st.subheader("📧 Contact Information")
    email = st.text_input("Email address")
    subscribe = st.checkbox("Subscribe to news and offers")
    
    # 🚚 Delivery Details
    st.subheader("🚚 Delivery Details")
    country = st.selectbox("Country/Region", ["Singapore", "Malaysia", "Indonesia"])
    name = st.text_input("Recipient Name")
    address = st.text_area("Address")
    postal = st.text_input("Postal Code")
    phone = st.text_input("Phone Number")
    save_info = st.checkbox("Save this information for future use")
    
    st.caption("Enter your shipping address to view available shipping methods.")
    
    # 💳 Payment Method
    st.subheader("💳 Payment Method")
    payment_method = st.radio(
        "Choose your payment option:",
        ["Credit/Debit Card", "PayNow (QR)", "Cash on Delivery"]
    )
    
    if payment_method == "Credit/Debit Card":
        saved_card = mylibrary.load_saved_card()
        card = mylibrary.render_card_payment(saved_card)

    
    elif payment_method == "PayNow (QR)":
        st.info("Scan the QR code below to pay via PayNow")
        st.image("images/paynowQR.jpg", caption="Scan to pay via PayNow")
        st.write(f"Amount to pay: ${total:.2f}")
    
    elif payment_method == "Cash on Delivery":
        st.info("Please prepare exact change. Delivery personnel will collect payment.")
    
    # 🛍️ Place Order Button
    if st.button("🛍️ Place Order"):
        st.success("✅ Your order has been placed successfully!")
        st.balloons()

Overwriting payment.py
